In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.3.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool
import com.google.adk.tools.AgentTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

---
## 🤔 Section 2: Why Multi-Agent Systems? + Your First Multi-Agent

**The Problem: The "Do-It-All" Agent**

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

**The Solution: A Team of Specialists**

Instead of one "do-it-all" agent, we can build a **multi-agent system**. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent *only* does research, another *only* writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

To learn more, check out the documentation related to [LLM agents in ADK](https://google.github.io/adk-docs/agents/llm-agents/).

**Architecture: Single Agent vs Multi-Agent Team**

```
graph TD
        subgraph Single["❌ Monolithic Agent"]
A["One Agent Does Everything"]
end

subgraph Multi["✅ Multi-Agent Team"]
B["Root Coordinator"] -- > C["Research Specialist"]
B -- > E["Summary Specialist"]

C -- >|findings| F["Shared State"]
E -- >|summary| F
        end

style A fill:#ffcccc
style B fill:#ccffcc
style F fill:#ffffcc
```

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

### 2.1 Example: Research & Summarization System

Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
        2. **Summarizer Agent** - Creates concise summaries from research findings

In [6]:
val researchAgent = LlmAgent.builder()
    .name("research_agent")
    .model(geminiModel)
    .description("A researcher agent")
    .instruction("""You are a specialized research agent. Your only job is to use the
     google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""")
    .tools(GoogleSearchTool.INSTANCE)
    .outputKey("research_findings")
    .build()

println("✅ Research agent created.")

✅ Research agent created.


In [7]:
val summarizerAgent = LlmAgent.builder()
    .name("summarizer_agent")
    .model(geminiModel)
    .description("A summarizer agent")
    .instruction("""Read the provided research findings: {research_findings}
     Create a concise summary as a bulleted list with 3-5 key points.""")
    .outputKey("final_summary")
    .build()

println("✅ Summarizer agent created.")

✅ Summarizer agent created.


Refer to the ADK documentation for more information on [guiding agents with clear and specific instructions](https://google.github.io/adk-docs/agents/llm-agents/).

Then we bring the agents together under a root agent, or coordinator:

In [8]:
val rootAgent = LlmAgent.builder()
    .name("research_coordinator")
    .model(geminiModel)
    .description("A model that coordinates other agents for research")
    .instruction("""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `research_agent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `summarizer_agent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""")
    .instruction("""You are a helpful assistant. Use Google Search for current info or if unsure""")
    .tools(AgentTool.create(researchAgent), AgentTool.create(summarizerAgent))
    .build()

println("✅ Root agent created.")

✅ Root agent created.


Here we're using `AgentTool` to wrap the sub-agents to make them callable tools for the root agent. We'll explore `AgentTool` in-detail on Day 2.

Let's run the agent and ask it about a topic:

In [9]:
var runner: InMemoryRunner = InMemoryRunner(rootAgent, "Research Flow")

In [10]:
var prompt = "What are the latest advancements in quantum computing and what do they mean for AI?"

var userMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [11]:
var session: Session? = runner
    .sessionService()
    .createSession("Research Flow", "user")
    .blockingGet()

In [40]:
var researchText = ""
var events: Flowable<Event> = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach{
    // println(event.stringifyContent())
        event -> event
            .content().get()
            .parts().get()
            .forEach {
                if (it.text().isPresent) {
                    researchText += it.text().get()
                }
            }
}
MIME(
    "text/markdown" to researchText
)

Agent response ->


Quantum computing advancements are set to significantly boost AI capabilities, leading to what's known as "Quantum AI." Here's a breakdown of the latest developments and their implications:

*   **Faster Processing:** Quantum computers can process information at speeds far beyond classical computers. This is a game-changer for AI and machine learning tasks like pattern recognition and data analysis. New quantum algorithms are being developed that promise much faster and more accurate AI decision-making.
*   **Better Optimization:** Quantum computing excels at solving complex optimization problems, which are fundamental to many AI applications. This could lead to more efficient AI in areas like resource allocation, predictive modeling, and recommendation systems.
*   **Advanced Generative Models:** Quantum circuits can improve generative AI models, potentially leading to breakthroughs in areas like designing new molecules or generating synthetic data.
*   **Hardware Improvements:** Researchers are making strides in creating more stable qubits, increasing their numbers, and extending their coherence times. This includes progress in areas like logical qubits and advancements in superconducting and photonic quantum systems.
*   **Hybrid Approaches:** In the near future, AI will likely run on a combination of quantum and classical computers, utilizing the strengths of both for optimal performance.

These advancements have the potential to revolutionize various sectors, including healthcare, finance, logistics, and autonomous systems. While there are still challenges to overcome in hardware development and integration, the synergy between quantum computing and AI is expected to redefine the future of problem-solving.

You've just built your first multi-agent system! You used a single "coordinator" agent to manage the workflow, which is a powerful and flexible pattern.

‼️ However, **relying on an LLM's instructions to control the order can sometimes be unpredictable.** Next, we'll explore a different pattern that gives you guaranteed, step-by-step execution.

---

## 🚥 Section 3: Sequential Workflows - The Assembly Line

**The Problem: Unpredictable Order**

The previous multi-agent system worked, but it relied on a **detailed instruction prompt** to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

**The Solution: A Fixed Pipeline**

        When you need tasks to happen in a **guaranteed, specific order**, you can use a `SequentialAgent`. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

**Use Sequential when:** Order matters, you need a linear pipeline, or each step builds on the previous one.

To learn more, check out the documentation related to [sequential agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/).

**Architecture: Blog Post Creation Pipeline**

```
graph LR
        A["User Input: Blog about AI"] -- > B["Outline Agent"]
B -- >|blog_outline| C["Writer Agent"]
C -- >|blog_draft| D["Editor Agent"]
D -- >|final_blog| E["Output"]

style B fill:#ffcccc
style C fill:#ccffcc
style D fill:#ccccff
```

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

### 3.1 Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
        3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [14]:
val outlineAgent = LlmAgent.builder()
    .name("outline_agent")
    .model(geminiModel)
    .description("A model that writes blog outlines")
    .instruction("""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""")
    .outputKey("blog_outline")
    .build()

println("✅ Outline agent created.")

✅ Outline agent created.


In [15]:
val writerAgent = LlmAgent.builder()
    .name("writer_agent")
    .model(geminiModel)
    .description("A model that writes a blog based on outlines")
    .instruction("""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""")
    .outputKey("blog_draft")
    .build()

println("✅ Writer agent created.")

✅ Writer agent created.


In [16]:
val editorAgent = LlmAgent.builder()
    .name("editor_agent")
    .model(geminiModel)
    .description("A model that checks text created by a writer agent")
    .instruction("""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""")
    .outputKey("final_blog")
    .build()

println("✅ Writer agent created.")

✅ Writer agent created.


In [17]:
val blogRootAgent = SequentialAgent.builder()
    .name("blog_pipeline_coordinator")
    .subAgents(outlineAgent, writerAgent, editorAgent)
    .build()

println("✅ Sequential agent created.")

✅ Sequential agent created.


In [18]:
var pipelineRunner: InMemoryRunner = InMemoryRunner(blogRootAgent, "Blog Pipeline Flow")

In [19]:
var pipelinePrompt = "Write a blog post about the benefits of multi-agent systems for software developers"

In [20]:
var startMsg: Content? = Content.fromParts(Part.fromText(pipelinePrompt))

In [27]:
var pipelineSession: Session? = pipelineRunner
    .sessionService()
    .createSession("Blog Pipeline Flow", "agency")
    .blockingGet()

In [42]:
var resultText = ""
var pipelineEvents: Flowable<Event> = pipelineRunner.runAsync("agency", pipelineSession.id(), startMsg)
println("Pipeline agent response ->")
pipelineEvents.blockingForEach{
    // println(event.stringifyContent())
    event -> event
        .content().get()
        .parts().get()
        .forEach {
            if (it.text().isPresent) {
                resultText += it.text().get()
            }
        }
}
MIME(
    "text/markdown" to resultText
)

Pipeline agent response ->


## Level Up Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

Ever felt like you're juggling too many tasks and wrestling with complex codebases on your own? What if you could delegate, automate, and even collaborate with intelligent "digital teammates" to build better software, faster? Welcome to the exciting world of Multi-Agent Systems (MAS) – a paradigm shift that's empowering software developers like never before.

At its core, MAS involves independent, autonomous entities, or agents, that interact to achieve individual or collective goals. Think of them as specialized software robots working in concert, much like a highly efficient digital workforce ready to be deployed. Unlike traditional monolithic programs, MAS excels at tackling complex, dynamic, and distributed problems that can be challenging for single programs to handle.

### Supercharging Development Efficiency

The benefits for developers are immense. MAS can **supercharge efficiency** by automating task distribution. Agents can be assigned specific responsibilities like code generation, testing, bug detection, and documentation, freeing human developers to focus on higher-level thinking and strategic problem-solving. Prototyping and iteration also accelerate, as MAS can rapidly generate and test multiple design variations or feature implementations, significantly speeding up the feedback loop. Imagine intelligent agents not just suggesting code, but intelligently refactoring existing codebases, identifying inefficiencies, and proposing optimizations based on learned patterns.

### Enhancing Software Quality and Robustness

Beyond speed, MAS significantly **enhances software quality and robustness**. These systems can proactively detect bugs by continuously monitoring code, analyzing dependencies, and simulating user behavior to identify potential issues *before* they reach production. They enable the creation of self-healing and adaptive systems that monitor performance, detect anomalies, and can even automatically repair parts of the software in real-time, leading to more resilient applications. Furthermore, MAS can be leveraged to create sophisticated testing environments and generate diverse test cases, performing more comprehensive validation of software functionality.

### The Collaborative Power of MAS

MAS also fosters **collaborative power within development teams**. Agents can act as intelligent intermediaries, bridging communication gaps and summarizing progress. They facilitate knowledge sharing by learning from each other's experiences and the collective codebase, creating a shared knowledge base that benefits the entire team. This human-agent collaboration allows developers to tackle incredibly complex software engineering challenges by leveraging the unique strengths of both humans and intelligent agents.

Multi-Agent Systems are no longer a futuristic concept; they are a tangible tool set poised to transform how we build software. By embracing the power of intelligent digital collaboration, developers can unlock unprecedented levels of efficiency, quality, and innovation. The question isn't *if* MAS will change software development, but *how quickly* you'll adopt this game-changing technology.## Level Up Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

Ever felt like you're juggling too many tasks and wrestling with complex codebases on your own? What if you could delegate, automate, and even collaborate with intelligent "digital teammates" to build better software, faster? Welcome to the exciting world of Multi-Agent Systems (MAS) – a paradigm shift that's empowering software developers like never before.

At its core, MAS involves independent, autonomous entities, or agents, that interact to achieve individual or collective goals. Think of them as specialized software robots working in concert, much like a highly efficient digital workforce ready to be deployed. Unlike traditional monolithic programs, MAS excels at tackling complex, dynamic, and distributed problems that can be challenging for single programs to handle.

### Supercharging Development Efficiency

The benefits for developers are immense. MAS can **supercharge efficiency** by automating task distribution. Agents can be assigned specific responsibilities like code generation, testing, bug detection, and documentation, freeing human developers to focus on higher-level thinking and strategic problem-solving. Prototyping and iteration also accelerate, as MAS can rapidly generate and test multiple design variations or feature implementations, significantly speeding up the feedback loop. Imagine intelligent agents not just suggesting code, but intelligently refactoring existing codebases, identifying inefficiencies, and proposing optimizations based on learned patterns.

### Enhancing Software Quality and Robustness

Beyond speed, MAS significantly **enhances software quality and robustness**. These systems can proactively detect bugs by continuously monitoring code, analyzing dependencies, and simulating user behavior to identify potential issues *before* they reach production. They enable the creation of self-healing and adaptive systems that monitor performance, detect anomalies, and can even automatically repair parts of the software in real-time, leading to more resilient applications. Furthermore, MAS can be leveraged to create sophisticated testing environments and generate diverse test cases, performing more comprehensive validation of software functionality.

### The Collaborative Power of MAS

MAS also fosters **collaborative power within development teams**. Agents can act as intelligent intermediaries, bridging communication gaps and summarizing progress. They facilitate knowledge sharing by learning from each other's experiences and the collective codebase, creating a shared knowledge base that benefits the entire team. This human-agent collaboration allows developers to tackle incredibly complex software engineering challenges by leveraging the unique strengths of both humans and intelligent agents.

Multi-Agent Systems are no longer a futuristic concept; they are a tangible tool set poised to transform how we build software. By embracing the power of intelligent digital collaboration, developers can unlock unprecedented levels of efficiency, quality, and innovation. The question isn't *if* MAS will change software development, but *how quickly* you'll adopt this game-changing technology.## Level Up Your Code: How Multi-Agent Systems Are Revolutionizing Software Development

Ever felt like you're juggling too many tasks and wrestling with complex codebases on your own? What if you could delegate, automate, and even collaborate with intelligent "digital teammates" to build better software, faster? Welcome to the exciting world of Multi-Agent Systems (MAS) – a paradigm shift that's empowering software developers like never before.

At its core, MAS involves independent, autonomous entities, or agents, that interact to achieve individual or collective goals. Think of them as specialized software robots working in concert, much like a highly efficient digital workforce ready to be deployed. Unlike traditional monolithic programs, MAS excels at tackling complex, dynamic, and distributed problems that can be challenging for single programs to handle.

### Supercharging Development Efficiency

The benefits for developers are immense. MAS can **supercharge efficiency** by automating task distribution. Agents can be assigned specific responsibilities like code generation, testing, bug detection, and documentation, freeing human developers to focus on higher-level thinking and strategic problem-solving. Prototyping and iteration also accelerate, as MAS can rapidly generate and test multiple design variations or feature implementations, significantly speeding up the feedback loop. Imagine intelligent agents not just suggesting code, but intelligently refactoring existing codebases, identifying inefficiencies, and proposing optimizations based on learned patterns.

### Enhancing Software Quality and Robustness

Beyond speed, MAS significantly **enhances software quality and robustness**. These systems can proactively detect bugs by continuously monitoring code, analyzing dependencies, and simulating user behavior to identify potential issues *before* they reach production. They enable the creation of self-healing and adaptive systems that monitor performance, detect anomalies, and can even automatically repair parts of the software in real-time, leading to more resilient applications. Furthermore, MAS can be leveraged to create sophisticated testing environments and generate diverse test cases, performing more comprehensive validation of software functionality.

### The Collaborative Power of MAS

MAS also fosters **collaborative power within development teams**. Agents can act as intelligent intermediaries, bridging communication gaps and summarizing progress. They facilitate knowledge sharing by learning from each other's experiences and the collective codebase, creating a shared knowledge base that benefits the entire team. This human-agent collaboration allows developers to tackle incredibly complex software engineering challenges by leveraging the unique strengths of both humans and intelligent agents.

Multi-Agent Systems are no longer a futuristic concept; they are a tangible tool set poised to transform how we build software. By embracing the power of intelligent digital collaboration, developers can unlock unprecedented levels of efficiency, quality, and innovation. The question isn't *if* MAS will change software development, but *how quickly* you'll adopt this game-changing technology.

👏 Great job! You've now created a reliable "assembly line" using a sequential agent, where each step runs in a predictable order.

**This is perfect for tasks that build on each other, but it's slow if the tasks are independent.** Next, we'll look at how to run multiple agents at the same time to speed up your workflow.